In [1]:
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim import corpora, models
from pprint import pprint
from collections import Counter

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    '''Lemmatizes and stems input word'''
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    """Lemmatizes and stems all input texts"""
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [4]:
with open("data/econ3_8.txt") as f:
    doc1 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [5]:
with open("data/econ3_7.txt") as f:
    doc2 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_doc)

In [ ]:
doc.split(".")

In [ ]:
sentences = []
curr = None
for sent in doc.split("."):
    if re.search(r"([:])", sent):
        if curr is not None:
            sentences.append(curr)
        curr = sent
    else:
        curr += "."+sent
sentences

In [6]:
# TODO filter meaningless sentences
def get_sents(doc):
    return doc.split(".")
def get_teacher_sents(doc):
    "Get sentences spoken by teacher. Works for prefaced name format"
    matches = re.findall(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)
    splits = re.split(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)[1:]
    teacher = Counter(matches).most_common(1)[0][0]
    return [splits[i] for i in range(len(splits)) if matches[i] == teacher]

In [7]:
get_teacher_sents(doc1)

[' Good morning. ',
 " Haha, you're, you're gone. Looks like I feel ",
 " I'm only hiding my tiredness behind my makeup. ",
 " When I woke up this morning. I said, oh, if I didn't have that conference, I need to go to later in Catholic, I would have canceled today. ",
 " You know, I'm going to think I'm, I'm going to see if I can miss a second conference so that after the second mid term, we don't have to ",
 " It's so much more exhausting than an actual midterm, at least for us, is it for you guys. ",
 " No. Okay, so, so it might be just because what we were just fielding all these emails this issue and that. So, actually. Yeah, so it's probably it might have been less exhausting for you guys because ",
 " You're in the quiet of your own place instead of a big room. So that's good. I guess so. There are some advantages, other than worrying about the technical stuff to being able to take exams online. That's interesting. ",
 " For work. We're going to do some polls and stuff. But you k

In [ ]:
get_sents(doc2)